In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import matplotlib

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cbook as cbook
import numpy as np
import pandas as pd
from decimal import Decimal
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import stats, interpolate
import scipy
import warnings
import datetime as dt
import landau
from multiprocessing import Pool, cpu_count

import importlib
import os
from tqdm.auto import tqdm

# local imports
from lib.constants import *

from datetime import datetime


from scipy.integrate import quad, tplquad
from scipy.special import erf
from scipy.interpolate import CubicSpline

In [ ]:
from run_times import run2date

In [ ]:
dosave = False
savedir = "plots_6_7_25/"
plt.rcParams.update({'font.size': 14})
plotqual = "Data%i"
plottitle = "Data Run %i"
tpcnames = ["EE", "EW", "WE", "WW"]
MAKEPLT = False

filedirs = [
    "/exp/icarus/data/users/gputnam/ICARUS-calibration-2D/validation/FNAL/",
]

outdir = "/exp/icarus/data/users/gputnam/ICARUS-calibration-2D/validation/FNAL/"

isMC = False
savedata = True
outf = outdir + "lifetimes_validation.txt"

In [ ]:
files = sum([[filedir + f for f in os.listdir(filedir) if f.endswith(".df") and 
              "etau" in f] for filedir in filedirs], [])
files

In [ ]:
run2file = {}

for f in files:
    run = int("".join([s for s in f.split("/")[-1].split(".")[0] if s.isdigit()]))
    
    if run not in run2file:
        run2file[run] = []
    
    run2file[run].append(f)

In [ ]:
runs = sorted(run2file.keys())

In [ ]:
runs

In [ ]:
NCHUNK = 10

In [ ]:
def landau_gaus(X, *p):
    mpv, eta, sigma, A = p
    sigma = np.minimum(sigma, 100*eta)
    return landau.landau.gauss_landau(X, mpv, eta, sigma, A)


In [ ]:
tbins = np.linspace(0, 1000, 21)
tbins = tbins[:-1]
tcenters = (tbins[1:] + tbins[:-1])/2.

qbins = np.linspace(100, 4000, 39*4+1)
qbin_centers = (qbins[1:] + qbins[:-1]) / 2.

In [ ]:
RUNS = {}
TAUS = {}
TAU_ERRS = {}

for tpc in tpcnames:
    RUNS[tpc] = []
    TAUS[tpc] = []
    TAU_ERRS[tpc] = []

In [ ]:
def lifetime(t, Q0, tau):
    return Q0*np.exp(-t/tau)

l_p0 = [500, 5e3]
l_bounds = ([0.0, 0.0], [1500., 20e3])

In [ ]:
class LandauData:
    def __init__(self, df, when, makeplot=False):
        self.df = df 
        self.when = when
        self.makeplot = makeplot
        
    def get_landau_data(self, blo, bhi, fig=None, title="", saveplt=""):        
        df = self.df
        when = self.when
        dqdx = df.dqdx
        
        thiswhen = when & (df.thitp < bhi) & (df.thitp > blo)
        N,_ = np.histogram(dqdx[thiswhen], bins=qbins)

        maxbin = np.argmax(N)
        when_fit = np.abs(np.array(range(len(qbin_centers))) - maxbin) < 15 # within 15 bins

        mpv_lo = 500
        mpv_hi = 1000
        mpv0 = 750
        p0 = [mpv0, 50, 50, np.max(N)*400.]
        bounds = ([mpv_lo, 10, 10, np.max(N)*400./100.], [mpv_hi, 250, 250, np.max(N)*400.*100.])
        if np.max(N) == 0:
            return None

        popt, perr = curve_fit(landau_gaus, qbin_centers[when_fit], N[when_fit], 
                               p0=p0, maxfev=10_000, sigma=np.maximum(np.sqrt(N),1)[when_fit], bounds=bounds)

        MPV = popt[0] - 0.228*popt[1]
        MPV_err = np.sqrt(np.diag(perr)[0] + (0.228**2)*np.diag(perr)[1] - 0.228*(perr[0,1] + perr[1,0]))

        loc = popt[0]
        loc_err = np.sqrt(np.diag(perr)[0])
        eta = popt[1]
        sigma = popt[2]
        
        if self.makeplot and fig:
            fig.hist(qbin_centers, weights=N, bins=qbins, label="Data")
            fig.plot(qbin_centers, landau_gaus(qbin_centers, *popt), label="Landau+Gaus Fit")
            fig.text(0.4, 0.25, "%.0f < $t_\\mathrm{hit}$ < %.0f $\\mu$s\n%.0f < 10-Wire Pitch < %.0f cm\n\nMPV = %.2f ADDC/cm\n$\\eta$ = %.2f ADDC/cm\n$\\sigma$ = %.2f ADDC/cm" %
                (blo, bhi, 0.3*NCHUNK, 1*NCHUNK, MPV, eta, sigma),
                    fontsize=14, transform=fig.gca().transAxes)
            fig.xlabel("dQ/dx [ADDC/cm]")
            fig.ylabel("# Depositions")
            fig.title(title)
            fig.xlim([200, 2500])
            fig.legend()
            fig.tight_layout()
            if saveplt and dosave:
                fig.savefig(saveplt)
                


        return MPV, MPV_err, loc, loc_err, eta, sigma

In [ ]:
def process_run(run, makeplt=MAKEPLT):
    iplt = 0
    # print(run)
    thisfiles = run2file[run]
    
    dfs = [pd.read_hdf(f) for f in thisfiles]
    
    for perhit_df in dfs:
        perhit_df["tpcEE"] = perhit_df.tpcE & (perhit_df.cryostat == 0)
        perhit_df["tpcEW"] = perhit_df.tpcW & (perhit_df.cryostat == 0)
        perhit_df["tpcWE"] = perhit_df.tpcE & (perhit_df.cryostat == 1)
        perhit_df["tpcWW"] = perhit_df.tpcW & (perhit_df.cryostat == 1)

        perhit_df["dqdx"] = perhit_df.charge / perhit_df.pitch
        perhit_df["thitp"] = (perhit_df.time * tick_period - perhit_df.pandora_t0 - tanode*tick_period) / 1000.

    for i, (f, df) in enumerate(zip(files, dfs)):
        if "bnb" in f:
            df["bnb"] = True
        else:
            df["bnb"] = False

        df["ifile"] = i

        if isMC:
            dfs[i] = dfs[i][~np.isnan(dfs[i].pandora_t0)]
            
    data = pd.concat(dfs)

    data = data.reset_index()
    data["entry"] = data.groupby(["entry", "ifile"]).ngroup()
    data = data.set_index(["entry", "chunk"])
    data = data.sort_index()
        
    perhit_df = data[data.run == run]
    Means = {}
    MPVs = {}
    MPV_errs = {}
    locs = {}
    loc_errs = {}

    etas = {}
    sigmas = {}
    
    TAUS = {}
    TAU_ERRS = {}

    for tpc in tpcnames:
        Means[tpc] = []
        MPVs[tpc] = []
        MPV_errs[tpc] = []
        locs[tpc] = []
        loc_errs[tpc] = []
        etas[tpc] = []
        sigmas[tpc] = []
        
    thisrun = int(perhit_df.run.unique()[0])

    runplotqual = plotqual % thisrun
    runplottitle = plottitle % thisrun
        
    when = (perhit_df.pitch < 1*NCHUNK)
    tpcs = [perhit_df.tpcEE, perhit_df.tpcEW, perhit_df.tpcWE, perhit_df.tpcWW]
    
    goodrun = True
    for tpc, tname in zip(tpcs, tpcnames):
        if not goodrun: 
            break
        calc = LandauData(perhit_df, when & tpc, makeplt)
        for blo, bhi in zip(tbins[:-1], tbins[1:]):
            if makeplt: plt.figure(iplt)

            x = calc.get_landau_data(blo, bhi, plt, runplottitle + " TPC %s" % tname, savedir + "%iwire_dqdx_thit%.0f_%.0f_TPC%s%s.pdf" % (NCHUNK, blo, bhi, tname, runplotqual))
            if x is None:
                goodrun = False
                break

            MPV, MPV_err, loc, loc_err, eta, sigma = x
            MPVs[tname].append(MPV)
            MPV_errs[tname].append(MPV_err)
            locs[tname].append(loc)
            loc_errs[tname].append(loc_err)
            etas[tname].append(eta)
            sigmas[tname].append(sigma)
            
            iplt += 1

    if not goodrun:
        return TAUS, TAU_ERRS, run
            
    for tpc in tpcnames:
        Means[tpc] = np.array(Means[tpc])
        MPVs[tpc] = np.array(MPVs[tpc])
        MPV_errs[tpc] = np.array(MPV_errs[tpc])
        locs[tpc] = np.array(locs[tpc])
        loc_errs[tpc] = np.array(loc_errs[tpc])
        etas[tpc] = np.array(etas[tpc])
        sigmas[tpc] = np.array(sigmas[tpc])
    
    whent = (tcenters > 100) & (tcenters < 900)
    # whent = (tcenters > 200) & (tcenters < 700)
    
    for i,tpc in enumerate(tpcnames):
        iplt += 1

        popt, perr = curve_fit(lifetime, tcenters[whent], MPVs[tpc][whent], 
                           p0=l_p0, maxfev=10_000, sigma=MPV_errs[tpc][whent], bounds=l_bounds)
        TAU, TAU_ERR = (popt[1], np.sqrt(np.diag(perr))[1])
        RUN = perhit_df.run.unique()[0]
        
        TAUS[tpc] = TAU
        TAU_ERRS[tpc] = TAU_ERR
        
        if makeplt:
            plt.figure(iplt)
            plt.errorbar(tcenters[whent], MPVs[tpc][whent], yerr=MPV_errs[tpc][whent], label="Data")
            plt.plot(tcenters[whent], lifetime(tcenters[whent], *popt), label="Fit")
            plt.text(0.05, 0.05, "$\\tau = %.0f \\pm %.0f \\mu$s" % (popt[1], np.sqrt(np.diag(perr))[1]),
                    fontsize=16, transform=plt.gca().transAxes)

            plt.xlabel("Hit Time [$\\mu$s]")
            plt.ylabel("MPV dQ/dx [ADDC/cm]")
            plt.title(runplottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
            plt.tight_layout()
            if dosave: plt.savefig(savedir + "%iwire_MPVdqdx_time_TPC%s%s.pdf" % (NCHUNK, tpc, runplotqual))
            

    if makeplt:          
        for i, tpc in enumerate(tpcnames):
            plt.figure(iplt)
            iplt += 1
            plt.plot(tcenters[whent], sigmas[tpc][whent])
            plt.xlabel("Hit Time [$\\mu$s]")
            plt.ylabel("Fit $\\sigma$ [ADDC/cm]")
            plt.title(runplottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
            plt.tight_layout()
            if dosave: plt.savefig(savedir + "%iwire_sigma_time_TPC%s%s.pdf" % (NCHUNK, tpc, runplotqual))

        for i, tpc in enumerate(tpcnames):
            plt.figure(iplt)
            iplt += 1
            plt.plot(tcenters[whent], etas[tpc][whent])
            plt.xlabel("Hit Time [$\\mu$s]")
            plt.ylabel("Fit $\\eta$ [ADDC/cm]")
            plt.title(runplottitle + " TPC " + tpc + " %i-Wire" % NCHUNK)
            plt.tight_layout()
            if dosave: plt.savefig(savedir + "%iwire_eta_time_TPC%s%s.pdf" % (NCHUNK, tpc, runplotqual))
        if MAKEPLT: plt.close("all")
        
    return TAUS, TAU_ERRS, run

In [ ]:
process_run(runs[0])

In [ ]:
TAUS = {}
TAU_ERRS = {}
RUNS = {}

for tpc in tpcnames:
    TAUS[tpc] = []
    TAU_ERRS[tpc] = []
    RUNS[tpc] = []
    
# can multiprocess if not making plots
if not MAKEPLT:
    with Pool(processes=12) as pool:
        for t, terr, r in tqdm(pool.imap_unordered(process_run, runs), total=len(runs)):
            for tpc in tpcnames:
                if tpc in t and t[tpc]:
                    TAUS[tpc].append(t[tpc])
                    TAU_ERRS[tpc].append(terr[tpc])
                    RUNS[tpc].append(r)
else:
    for t, terr, r in tqdm(map(process_run, runs), total=len(runs)):
        for tpc in tpcnames:
            if tpc in t and t[tpc]:
                TAUS[tpc].append(t[tpc])
                TAU_ERRS[tpc].append(terr[tpc])
                RUNS[tpc].append(r)

In [ ]:
for tpc in tpcnames:
    # RUNS[tpc], TAUS[tpc], TAU_ERRS[tpc] = list(zip(*sorted(zip(RUNS[tpc], TAUS[tpc], TAU_ERRS[tpc]))))
    RUNS[tpc] = np.array(RUNS[tpc])
    TAUS[tpc] = np.array(TAUS[tpc])
    TAU_ERRS[tpc] = np.array(TAU_ERRS[tpc])

In [ ]:
TAU_ERRS

In [ ]:
for tpc in tpcnames:
    good = TAU_ERRS[tpc] / TAUS[tpc] < 0.05
    plt.errorbar(RUNS[tpc][good], TAUS[tpc][good], TAU_ERRS[tpc][good], label=tpc, 
                 linestyle="none", marker=".")
plt.xlabel("Run Number")
plt.ylabel("Electron Lifetime [$\\mu$s]")
plt.title("ICARUS Data")
plt.legend(ncol=2, loc="upper left")
plt.tight_layout()
# plt.ylim([plt.ylim()[0], plt.ylim()[1]+400])

if dosave: plt.savefig(savedir + "%iwire_etau_perrun.pdf" % (NCHUNK,))

In [ ]:
datedruns = np.array([r in run2date for r in RUNS["EE"]])

In [ ]:
RUNS["EE"][datedruns]

In [ ]:
for tpc in tpcnames:
    good = TAU_ERRS[tpc] / TAUS[tpc] < 0.05
    plt.errorbar([run2date[int(r)] for r in RUNS[tpc][good & datedruns]], TAUS[tpc][good & datedruns], TAU_ERRS[tpc][good & datedruns], 
                 label=tpc, linestyle="none", marker=".")
plt.xlabel("Run Date")
plt.xticks(rotation=30, ha='right')

plt.ylabel("Electron Lifetime [$\\mu$s]")
plt.legend(ncol=4, loc='upper center', bbox_to_anchor=(0.5, 1.275))
# plt.ylim([plt.ylim()[0], plt.ylim()[1]+400])
plt.tight_layout()

if dosave: plt.savefig(savedir + "%iwire_etau_perdate.pdf" % (NCHUNK,))

In [ ]:
def correct_mc_bias(tau, bias=0.00e-3):
    return 1/(1/tau - bias)

In [ ]:
correct_mc_bias(2718)

In [ ]:
import matplotlib.ticker as plticker
import matplotlib.patches as patches

In [ ]:
plt.figure(0, figsize=(10, 4.8))

for tpc in tpcnames:
    good = TAU_ERRS[tpc] / TAUS[tpc] < 0.05
    plt.errorbar([run2date[r] for r in RUNS[tpc][good & datedruns]], correct_mc_bias(TAUS[tpc][good & datedruns]), 100, 
                 label=tpc, linestyle="none", marker=".")
plt.xlabel("Run Date")
plt.xticks(rotation=30, ha='right')
loc = plticker.MultipleLocator(base=2e3) # this locator puts ticks at regular intervals
plt.gca().yaxis.set_major_locator(loc)

plt.ylabel("Electron Lifetime [$\\mu$s]")

h, l = plt.gca().get_legend_handles_labels()
ph = [plt.plot([],marker="", ls="")[0]]
handles = ph + h
labels = ["TPC: "] + l
leg = plt.legend(handles, labels, ncol=5, 
               loc='upper center', bbox_to_anchor=(0.5, 1.2),columnspacing=1, handletextpad=0.4)
for vpack in leg._legend_handle_box.get_children()[:1]:
    for hpack in vpack.get_children():
        hpack.get_children()[0].set_width(0)

#l.get_title().set_position((-150, -20)) # -10 is a guess

plt.ylim([plt.ylim()[0], plt.ylim()[1]+400])
plt.tight_layout()

# rect = patches.Rectangle((0.1, 0.0), 0.1, 1, 
#                          linestyle="none", facecolor="pink",
#                         transform=plt.gca().transAxes)
# plt.gca().add_patch(rect)
# plt.text(0.21, 0.825, "Planned\nCryo. Work", transform=plt.gca().transAxes, color="deeppink")

plt.axvline([run1_start_date], ymin=0.05, ymax=0.6, color="black", linestyle=":")

plt.text(0.2, 0.35, "Run A",
        transform=plt.gca().transAxes, fontsize=14)

plt.text(0.4, 0.35, "Run 1",
        transform=plt.gca().transAxes, fontsize=14)

plt.text(0.7, 0.35, "Run 2",
        transform=plt.gca().transAxes, fontsize=14)

plt.text(0.5, 0.7, "Summer +\nTechnical\nShutdown",
        transform=plt.gca().transAxes, fontsize=14)

def lifetime_2_attenuate(tau, t=0.5e3):
    return 1-np.exp(-t/tau)

def attenuate_2_lifetime(a, t=0.5e3):
    return - t / np.log(1-a)

ax2 = plt.gca().secondary_yaxis("right", functions=(lifetime_2_attenuate, attenuate_2_lifetime))
ax2.locator_params(nbins=5, axis='y')
ax2.set_ylabel("Mean Signal Attenuation", rotation=270, labelpad=15)
plt.tight_layout()

if dosave: plt.savefig(savedir + "%iwire_etau_perdate_correct.pdf" % (NCHUNK,))

In [ ]:
if savedata:
    with open(outf, "w") as f:
        f.write("Run" + " " + " ".join(tpcnames) + "\n")
        for d in zip(list(RUNS["EE"][good]), *[list(TAUS[tpc][good]) for tpc in tpcnames]):
            f.write(" ".join(map(str, d)) + "\n")

In [ ]:
savedata